## Finding:
* 1.matrix factorization works better than item KNN
* 2.adding game information to MF models, but the RMSE is worse, needs to find reasons

In [1]:
import boto3
import pickle
import h5py
import tables
import numpy as np
import pandas as pd
import sklearn
import caserec

In [2]:
import turicreate as tc

In [3]:
actions = tc.SFrame.read_csv('training_data_3column.csv')
#items = tc.SFrame.read_csv('./dataset/ml-20m/movies.csv')

Finished parsing file /Users/shujdong/w210_game_recommendation/training_data_3column.csv

Parsing completed. Parsed 100 lines in 1.55192 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,float]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Read 2115061 lines. Lines per second: 1.13061e+06

Finished parsing file /Users/shujdong/w210_game_recommendation/training_data_3column.csv

Parsing completed. Parsed 12356262 lines in 5.44659 secs.

In [7]:
df=pd.read_csv('training_data_3column.csv')
df.head()

,Unnamed: 0,game_id,user_id,rating
0,0,0,Lil Red Head,7.0
1,1,0,hsjx945,8.0
2,2,0,RuffRyder,0.0
3,3,0,SkywalterDBZ,7.0
4,4,0,Olmestig,8.0


In [8]:
df.game_id.nunique()

21702

In [4]:
def recommendation_models(data,model):

    training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'user_id', 'game_id')
    ml = model(training_data, 'user_id', 'game_id',target='rating')
    evaluation = ml.evaluate(validation_data)
    #precision_recall =ml.evaluate_precision_recall(validation_data)
    return evaluation

In [5]:
recommendation_models(actions,tc.factorization_recommender.create)

Preparing data set.

Data has 12350437 observations with 403750 users and 21702 items.

Data prepared in: 12.3269s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Computing final objective value and training RMSE.

Final objective value: 4.3102

Final training RMSE: 2.07403


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.10554803788903923 | 0.028683155665560546 |
|   2    | 0.10081190798376186 | 0.051595308319377274 |
|   3    | 0.08615245827695084 |  0.0675024054751416  |
|   4    | 0.07746955345060899 | 0.08117966810855892  |
|   5    | 0.07361299052774022 | 0.09393420262908568  |
|   6    | 0.06765899864682001 | 0.10038080471133526  |
|   7    | 0.06069978735743282 | 0.10453778928708585  |
|   8    | 0.05700270635994587 | 0.11178666885020655  |
|   9    | 0.05322507893549842 |  0.1177071195032261  |
|   10   | 0.05128552097428959 | 0.12247753622342843  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 2.5673606203284605

Per User RMSE (best)
+----------+----------------------+-------+
| user_id  |         rmse         | count 

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 13302
 
 Data:
 +------------+--------+---------------------+--------------------+-------+
 |  user_id   | cutoff |      precision      |       recall       | count |
 +------------+--------+---------------------+--------------------+-------+
 | Majomaja92 |   1    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   2    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   3    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   4    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   5    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   6    | 0.16666666666666666 | 0.3333333333333333 |   3   |
 | Majomaja92 |   7    | 0.14285714285714285 | 0.3333333333333333 |   3   |
 | Majomaja92 |   8    |        0.125        | 0.3333333333333333 |   3   |
 | Majomaja92 |   9    |  0.11

In [6]:
recommendation_models(actions,tc.ranking_factorization_recommender.create)

Preparing data set.

Data has 12350437 observations with 403750 users and 21702 items.

Data prepared in: 12.1357s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Computing final objective value and training RMSE.

Final objective value: 9.91172

Final training RMSE: 2.62658


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.07577807848443847  | 0.012784878711062353 |
|   2    | 0.07104194857916103  | 0.03076711280176167  |
|   3    | 0.07126747857465042  | 0.048375584784518866 |
|   4    | 0.06529093369418133  | 0.05672860559960968  |
|   5    | 0.06143437077131261  |  0.068196006685172   |
|   6    | 0.05683355886332883  |  0.0740297065056656  |
|   7    | 0.05548037889039242  |  0.0824915399408574  |
|   8    | 0.05361975642760487  | 0.08933396111362717  |
|   9    | 0.05187189896256204  | 0.09334218416478947  |
|   10   | 0.049391069012178636 | 0.09854779892120422  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 3.2381507314460953

Per User RMSE (best)
+-----------+----------------------+-------+
|  user_id  |         rmse 

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 13302
 
 Data:
 +------------+--------+---------------------+--------------------+-------+
 |  user_id   | cutoff |      precision      |       recall       | count |
 +------------+--------+---------------------+--------------------+-------+
 | Majomaja92 |   1    |         1.0         | 0.3333333333333333 |   3   |
 | Majomaja92 |   2    |         0.5         | 0.3333333333333333 |   3   |
 | Majomaja92 |   3    |  0.3333333333333333 | 0.3333333333333333 |   3   |
 | Majomaja92 |   4    |         0.25        | 0.3333333333333333 |   3   |
 | Majomaja92 |   5    |         0.2         | 0.3333333333333333 |   3   |
 | Majomaja92 |   6    | 0.16666666666666666 | 0.3333333333333333 |   3   |
 | Majomaja92 |   7    | 0.14285714285714285 | 0.3333333333333333 |   3   |
 | Majomaja92 |   8    |        0.125        | 0.3333333333333333 |   3   |
 | Majomaja92 |   9    |  0.11

In [7]:
recommendation_models(actions,tc.item_similarity_recommender.create)


Warning: Ignoring columns X1;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 12350437 observations with 403750 users and 21702 items.

Data prepared in: 9.20791s

...

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 13.8085s


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.12990527740189436 | 0.028809272093688068 |
|   2    | 0.10487144790257104 | 0.046358681736989066 |
|   3    | 0.09427153811456924 | 0.06161427384408022  |
|   4    | 0.08728010825439779 |  0.0728899181353497  |
|   5    | 0.08119079837618406 | 0.08090949951642092  |
|   6    | 0.07668019846639607 | 0.09001232214195078  |
|   7    | 0.07133191571621882 | 0.09688218054232696  |
|   8    | 0.06867388362652231 | 0.10270734662637618  |
|   9    | 0.06600511201323109 | 0.11085839020757411  |
|   10   | 0.06414073071718539 | 0.11839622114048255  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 4.530671627038898

Per User RMSE (best)
+----------------+------+-------+
|    user_id     | rmse | count |
+----------------+-

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 13302
 
 Data:
 +------------+--------+---------------------+--------------------+-------+
 |  user_id   | cutoff |      precision      |       recall       | count |
 +------------+--------+---------------------+--------------------+-------+
 | Majomaja92 |   1    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   2    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   3    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   4    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   5    |         0.0         |        0.0         |   3   |
 | Majomaja92 |   6    | 0.16666666666666666 | 0.3333333333333333 |   3   |
 | Majomaja92 |   7    | 0.14285714285714285 | 0.3333333333333333 |   3   |
 | Majomaja92 |   8    |        0.125        | 0.3333333333333333 |   3   |
 | Majomaja92 |   9    |  0.11

In [8]:
recommendation_models(actions,tc.popularity_recommender.create)

Warning: Ignoring columns X1;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 12350437 observations with 403750 users and 21702 items.

Data prepared in: 9.48178s

12350437 observations to process; with 21702 unique items.


Precision and recall summary statistics by cutoff
+--------+-----------------------+------------------------+
| cutoff |     mean_precision    |      mean_recall       |
+--------+-----------------------+------------------------+
|   1    |          0.0          |          0.0           |
|   2    | 0.0020297699594046025 | 0.00011673036030275546 |
|   3    | 0.0018042399639151996 | 0.0001246436934778221  |
|   4    | 0.0016914749661705007 | 0.00017668907705229906 |
|   5    | 0.0016238159675236806 | 0.00021769453077764449 |
|   6    | 0.0013531799729364009 | 0.0002176945307776445  |
|   7    | 0.0011598685482312008 | 0.0002176945307776445  |
|   8    | 0.0015223274695534506 | 0.0003243146259043056  |
|   9    | 0.0034581265975041357 |  0.004269837040641878  |
|   10   |  0.003112313937753722 |  0.004269837040641878  |
+--------+-----------------------+------------------------+
[10 rows x 3 columns]


Overall RMSE: 3.391750085211376

Per User RMSE (best)
+----------+-------------------

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 13302
 
 Data:
 +------------+--------+-----------+--------+-------+
 |  user_id   | cutoff | precision | recall | count |
 +------------+--------+-----------+--------+-------+
 | Majomaja92 |   1    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   2    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   3    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   4    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   5    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   6    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   7    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   8    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   9    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   10   |    0.0    |  0.0   |   3   |
 +------------+--------+-----------+--------+-------+
 [13302 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, n

## Use factorization model to use side information when recommending items to new users (info. )

TODO:extract game info. add to factorization model as item info. to see if it improves the model performance


In [9]:
items = tc.SFrame.read_csv('game_info_full.csv')


Finished parsing file /Users/shujdong/w210_game_recommendation/game_info_full.csv

Parsing completed. Parsed 100 lines in 0.757016 secs.

------------------------------------------------------
Inferred types from first 100 line(s) of file as 
column_type_hints=[int,int,str,int,list,str,list,list]
If parsing fails due to incorrect types, you can correct
the inferred type list above and pass it to read_csv in
the column_type_hints argument
------------------------------------------------------


Finished parsing file /Users/shujdong/w210_game_recommendation/game_info_full.csv

Parsing completed. Parsed 108571 lines in 0.647931 secs.

In [10]:
items_final=items.dropna()

In [11]:
items_final.head()

X1,game_id,game_title,release_year,persons_duration_age_rating ...,publisher
0,1,Die Macher,1986,"[3–5, 240, 14+, 4.34]","[['Karl-Heinz Schmiel'],['Marcus Gschwendtner', ..."
1,2,Dragonmaster,1981,"[3–4, 30, 12+, 1.96]","[['G. W. ""Jerry""D'Arcey'], ['Bob ..."
2,3,Samurai,1998,"[2–4, 30–60, 10+, 2.50]","[['Reiner Knizia'],['Franz Vohwinkel'], ..."
3,4,Tal der Könige,1992,"[2–4, 60, 12+, 2.67]","[['Christian Beierer'],['Thomas di Paolo'], ..."
4,5,Acquire,1964,"[2–6, 90, 12+, 2.51]","[['Sid Sackson'], ['ScottOkumura', 'Peter ..."
5,6,Mare Mediterraneum,1989,"[2–6, 240, 12+, 3.00]","[['Jean du Poël'], ['Jeandu Poël'], ['Historien ..."
6,7,Cathedral,1978,"[2, 20, 8+, 1.81]","[[' \t\t\t\t\tCastleCity, ', ' ..."
7,8,Lords of Creation,1993,"[2–5, 120, 12+, 2.40]","[['Martin Wallace'],['Bryan King'], ..."
8,9,El Caballero,1998,"[2–4, 90, 13+, 3.19]","[['Wolfgang Kramer','Richard Ulrich'], ..."
9,10,Elfenland,1998,"[2–6, 60, 10+, 2.16]","[['Alan R. Moon'],['Doris Matthäus'], ..."


In [12]:
items_final.head()
items_final=items_final[['game_id','genre','label']]

In [13]:
def recommendation_models_game_info(action_data,game_info, model):

    training_data, validation_data = tc.recommender.util.random_split_by_user(action_data, 'user_id', 'game_id')
    ml = model(training_data, 'user_id', 'game_id',target='rating',item_data=game_info,side_data_factorization=True)
    evaluation = ml.evaluate(validation_data)
    #precision_recall =ml.evaluate_precision_recall(validation_data)
    return evaluation

In [14]:
recommendation_models_game_info(actions,items_final,tc.factorization_recommender.create)

Preparing data set.

Data has 12350437 observations with 403750 users and 102371 items.

Data prepared in: 11.208s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Computing final objective value and training RMSE.

Final objective value: 4.66592

Final training RMSE: 2.15898


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.06495263870094725  | 0.011394976098817053 |
|   2    | 0.05345060893098782  | 0.023087568752963483 |
|   3    | 0.041497519170049596 |  0.031973078822506   |
|   4    | 0.03721244925575102  | 0.03893666615483489  |
|   5    | 0.03410013531799725  | 0.04525830175204473  |
|   6    | 0.031123139377537218 | 0.05049975883669189  |
|   7    | 0.028996713705780004 | 0.05668334305463515  |
|   8    | 0.02621786197564276  | 0.05881782336908837  |
|   9    | 0.024206886182528916 | 0.06226697726784729  |
|   10   | 0.02273342354533151  | 0.06375074897809854  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 2.626382357603605

Per User RMSE (best)
+------------+----------------------+-------+
|  user_id   |         rmse

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 13302
 
 Data:
 +------------+--------+-----------+--------+-------+
 |  user_id   | cutoff | precision | recall | count |
 +------------+--------+-----------+--------+-------+
 | Majomaja92 |   1    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   2    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   3    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   4    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   5    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   6    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   7    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   8    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   9    |    0.0    |  0.0   |   3   |
 | Majomaja92 |   10   |    0.0    |  0.0   |   3   |
 +------------+--------+-----------+--------+-------+
 [13302 rows x 5 columns]
 Note: Only the head of the SFrame is printed.
 You can use print_rows(num_rows=m, n

In [15]:
recommendation_models_game_info(actions,items_final,tc.ranking_factorization_recommender.create)

Preparing data set.

Data has 12350437 observations with 403750 users and 102371 items.

Data prepared in: 11.8532s

Training ranking_factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Computing final objective value and training RMSE.

Final objective value: 4.79553

Final training RMSE: 1.96147


Precision and recall summary statistics by cutoff
+--------+----------------------+---------------------+
| cutoff |    mean_precision    |     mean_recall     |
+--------+----------------------+---------------------+
|   1    | 0.11366711772665765  | 0.03240800763287351 |
|   2    | 0.09472259810554802  | 0.05496512718357886 |
|   3    | 0.07758231844835368  |  0.0639933994846084 |
|   4    | 0.07104194857916105  | 0.07281525904858313 |
|   5    | 0.06468200270635995  |  0.0820830015456251 |
|   6    | 0.062246278755074415 | 0.09575542094191745 |
|   7    | 0.059153295959791206 |  0.1016393367329531 |
|   8    | 0.054972936400541274 | 0.10781495834321488 |
|   9    | 0.05277401894451958  |  0.1149469995934787 |
|   10   | 0.05087956698240866  | 0.11894820932683005 |
+--------+----------------------+---------------------+
[10 rows x 3 columns]


Overall RMSE: 2.834940526338545

Per User RMSE (best)
+----------------+----------------------+-------+
|    user_id     |         rmse      

{'precision_recall_by_user': Columns:
 	user_id	str
 	cutoff	int
 	precision	float
 	recall	float
 	count	int
 
 Rows: 13302
 
 Data:
 +------------+--------+-----------+--------------------+-------+
 |  user_id   | cutoff | precision |       recall       | count |
 +------------+--------+-----------+--------------------+-------+
 | Majomaja92 |   1    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   2    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   3    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   4    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   5    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   6    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   7    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   8    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   9    |    0.0    |        0.0         |   3   |
 | Majomaja92 |   10   |    0.1    | 0.3333333333333333 |   3   |
 +------

In [18]:
training_data, validation_data = tc.recommender.util.random_split_by_user(actions, 'user_id', 'game_id')
ml = tc.factorization_recommender.create(training_data, 'user_id', 'game_id',target='rating')
evaluation = ml.evaluate(validation_data)

Preparing data set.

Data has 12350437 observations with 403750 users and 21702 items.

Data prepared in: 12.5528s

Training factorization_recommender for recommendations.

+--------------------------------+--------------------------------------------------+----------+

...

Optimization Complete: Maximum number of passes through the data reached.

Computing final objective value and training RMSE.

Final objective value: 4.34013

Final training RMSE: 2.08096


Precision and recall summary statistics by cutoff
+--------+---------------------+----------------------+
| cutoff |    mean_precision   |     mean_recall      |
+--------+---------------------+----------------------+
|   1    | 0.10148849797023005 | 0.029954360286945373 |
|   2    | 0.09404600811907986 | 0.04836472247654015  |
|   3    | 0.08930987821380243 | 0.07178204416248764  |
|   4    | 0.07848443843031123 | 0.07921805623167678  |
|   5    | 0.06874154262516916 | 0.08330452507088636  |
|   6    | 0.06202074875958503 |  0.0876458828587503  |
|   7    | 0.05838005026097038 | 0.09390268882239748  |
|   8    | 0.05463464140730717 | 0.09905013937620953  |
|   9    | 0.05217260562321451 | 0.10595546491689263  |
|   10   | 0.04898511502029768 | 0.10873194800427974  |
+--------+---------------------+----------------------+
[10 rows x 3 columns]


Overall RMSE: 2.6066072970955165

Per User RMSE (best)
+-------------+----------------------+-------+
|   user_id   |         rmse         | 

In [40]:
actions.head()


X1,game_id,user_id,rating
0,0,Lil Red Head,7.0
1,0,hsjx945,8.0
2,0,RuffRyder,0.0
3,0,SkywalterDBZ,7.0
4,0,Olmestig,8.0
5,0,grassquit,9.0
6,0,erzengel,0.0
7,0,Feeble,0.0
8,0,thisisadamj,0.0
9,0,Tiberiusfox,9.0


In [46]:
#recommendations = ml.recommend(users=['ce'],items=['3'])

m = tc.item_similarity_recommender.create(actions,'user_id','game_id',target='rating',similarity_type='cosine')



Warning: Ignoring columns X1;

To use these columns in scoring predictions, use a model that allows the use of additional features.

Preparing data set.

Data has 12356262 observations with 403779 users and 21702 items.

Data prepared in: 16.8618s

...

| 49.55s                              | 100              | 21702           |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 50.4311s

In [53]:

similar_items = m.get_similar_items(k=5)
similar_items

game_id,similar,score,rank
0,3683,0.39362555742263794,1
0,8427,0.3672316074371338,2
0,414,0.36566251516342163,3
0,485,0.35234397649765015,4
0,4890,0.35048937797546387,5
3259,6107,0.2073848843574524,1
3259,2404,0.1597203016281128,2
3259,6149,0.15939772129058838,3
3259,198,0.14657926559448242,4
3259,67,0.14644795656204224,5


In [89]:
new_obs_data = tc.SFrame({'user_id' : ['chai', 'chai'],
                                'game_id' : ['13927', '2009']})
recommendations = m.recommend(['chai'], new_observation_data = new_obs_data)

#my_list_of_items=['6689','21311']
#m.recommend(my_list_of_items)

In [90]:
recommendations

user_id,game_id,score,rank
chai,720,0.23408204913139344,1
chai,2342,0.2317007315158844,2
chai,2738,0.23150307059288025,3
chai,12,0.23113720893859863,4
chai,417,0.21986159324645996,5
chai,8422,0.21902016043663025,6
chai,10,0.2186650538444519,7
chai,47,0.21175797462463378,8
chai,5686,0.20678720831871034,9
chai,39,0.20372353315353395,10


In [91]:
m.save('game_rec')